In [1]:
%load_ext autoreload
%autoreload 2

In [50]:
import os
import numpy as np
import matplotlib.pyplot as plt
from aws_utils import GoesAwsBucket, goes_proj, quick_plot
aws = GoesAwsBucket('noaa-goes16')

In [80]:
import itertools
def extract_patches( data, scale=100 ):
    """
    Given full disk GOES image, extracts patches of size 256x256 from the interior of the image.
    K is converted to C, and data is converted to int16 by multiplying by scale
    """
    i=range(900,4400,256) # bottom corners of the patches
    sz = 256 # length of patches in pixels
    arr = data['array'].data
    corners = list(itertools.product(i,i)) 
    N = len(corners)
    patches = np.zeros((N,sz,sz),dtype=np.int16)
    for k,(x0,y0) in enumerate(corners):
        patches[k,:,:] = np.int16(100*(arr[y0:y0+sz,x0:x0+sz]-273.15))
    return patches

def get_batch(gather_config,n_subsample=50):
    while True:
        time = random_time()
        patches_dict={}
        for k,g in gather_config.items():
            data = aws.get(g['product'],time,g.get('channel',''))
            if data is None:
                print('%s not found from time %s!' % (g['product'],time))
                stop=False
                break
            else:
                patches_dict[k]=extract_patches(data,scale=g.get('scale',100))
                stop=True
        if stop:
            break
    mask = np.random.choice( patches_dict[k].shape[0], n_subsample)
    for k in patches_dict:
        patches_dict[k] = patches_dict[k][mask,:,:]
    return patches_dict



In [57]:
gather_config = {
    'ACTP':{'product':'ABI-L2-ACTPF','scale':1},
    'ACM':{'product':'ABI-L2-ACMF','scale':1},
    'C07':{'product':'ABI-L2-CMIPF','channel':7},
    'C09':{'product':'ABI-L2-CMIPF','channel':9},
    'C13':{'product':'ABI-L2-CMIPF','channel':13},
    'C16':{'product':'ABI-L2-CMIPF','channel':16},
}

In [81]:
import datetime
def random_time(start_date=datetime.datetime(2019,12,1), # ABI-L2-ACMF only avail after ~dec 2019?
                end_date=datetime.datetime(2020,11,1)):
    d = (end_date-start_date)
    return (start_date + datetime.timedelta(seconds=np.random.randint(int(d.total_seconds()))))

In [77]:
b = get_batch(gather_config)

In [78]:
b['C07'].shape

(30, 256, 256)

In [ ]:
import h5py
# BE CAREFUL!!  This will overwrite the file if run again
output_file = 'G16_c09_c07_c13_c09_ACTP_ACM.h5'
#if os.path.exists(output_file):
#    raise ValueError('%s exists, please remove' % output_file)

# gather first batch
with h5py.File(output_file, 'w') as hf:
    patches=get_batch(gather_config)
    for k,g in gather_config.items():
        hf.create_dataset(k,data=patches[k],maxshape=(None,patches[k].shape[1],patches[k].shape[2]))
        
n_samples = 199
with h5py.File(output_file, 'a') as hf:
    for i in range(n_samples):
        print('----------------------')
        print('PROCESSING BATCH %d/%d' % (i,n_samples))
        print('----------------------')
        patches=get_batch(gather_config)
        for k,g in gather_config.items():
            hf[k].resize((hf[k].shape[0] + patches[k].shape[0]), axis = 0)
            hf[k][-patches[k].shape[0]:]  = patches[k]
        

----------------------
PROCESSING BATCH 0/199
----------------------
----------------------
PROCESSING BATCH 1/199
----------------------
ABI-L2-ACMF not found from time 2019-12-02 08:33:00!
----------------------
PROCESSING BATCH 2/199
----------------------
----------------------
PROCESSING BATCH 3/199
----------------------
----------------------
PROCESSING BATCH 4/199
----------------------
----------------------
PROCESSING BATCH 5/199
----------------------
----------------------
PROCESSING BATCH 6/199
----------------------
----------------------
PROCESSING BATCH 7/199
----------------------
----------------------
PROCESSING BATCH 8/199
----------------------
----------------------
PROCESSING BATCH 9/199
----------------------


----------------------
PROCESSING BATCH 10/199
----------------------
----------------------
PROCESSING BATCH 11/199
----------------------
----------------------
PROCESSING BATCH 12/199
----------------------
----------------------
PROCESSING BATCH 13/199
----------------------
----------------------
PROCESSING BATCH 14/199
----------------------
----------------------
PROCESSING BATCH 15/199
----------------------
----------------------
PROCESSING BATCH 16/199
----------------------
----------------------
PROCESSING BATCH 17/199
----------------------
----------------------
PROCESSING BATCH 18/199
----------------------
----------------------
PROCESSING BATCH 19/199
----------------------
----------------------
PROCESSING BATCH 20/199
----------------------


----------------------
PROCESSING BATCH 21/199
----------------------
----------------------
PROCESSING BATCH 22/199
----------------------
----------------------
PROCESSING BATCH 23/199
----------------------
----------------------
PROCESSING BATCH 24/199
----------------------
----------------------
PROCESSING BATCH 25/199
----------------------
----------------------
PROCESSING BATCH 26/199
----------------------
----------------------
PROCESSING BATCH 27/199
----------------------
----------------------
PROCESSING BATCH 28/199
----------------------
----------------------
PROCESSING BATCH 29/199
----------------------
----------------------
PROCESSING BATCH 30/199
----------------------
----------------------
PROCESSING BATCH 31/199
----------------------


----------------------
PROCESSING BATCH 32/199
----------------------
----------------------
PROCESSING BATCH 33/199
----------------------
----------------------
PROCESSING BATCH 34/199
----------------------
----------------------
PROCESSING BATCH 35/199
----------------------
----------------------
PROCESSING BATCH 36/199
----------------------
----------------------
PROCESSING BATCH 37/199
----------------------
----------------------
PROCESSING BATCH 38/199
----------------------
----------------------
PROCESSING BATCH 39/199
----------------------
----------------------
PROCESSING BATCH 40/199
----------------------
----------------------
PROCESSING BATCH 41/199
----------------------
----------------------
PROCESSING BATCH 42/199
----------------------


----------------------
PROCESSING BATCH 43/199
----------------------
----------------------
PROCESSING BATCH 44/199
----------------------
----------------------
PROCESSING BATCH 45/199
----------------------
----------------------
PROCESSING BATCH 46/199
----------------------
----------------------
PROCESSING BATCH 47/199
----------------------
----------------------
PROCESSING BATCH 48/199
----------------------
----------------------
PROCESSING BATCH 49/199
----------------------
----------------------
PROCESSING BATCH 50/199
----------------------
----------------------
PROCESSING BATCH 51/199
----------------------
----------------------
PROCESSING BATCH 52/199
----------------------
----------------------
PROCESSING BATCH 53/199
----------------------


----------------------
PROCESSING BATCH 54/199
----------------------
----------------------
PROCESSING BATCH 55/199
----------------------
----------------------
PROCESSING BATCH 56/199
----------------------
----------------------
PROCESSING BATCH 57/199
----------------------
----------------------
PROCESSING BATCH 58/199
----------------------
----------------------
PROCESSING BATCH 59/199
----------------------
----------------------
PROCESSING BATCH 60/199
----------------------
----------------------
PROCESSING BATCH 61/199
----------------------
----------------------
PROCESSING BATCH 62/199
----------------------
----------------------
PROCESSING BATCH 63/199
----------------------
ABI-L2-ACTPF not found from time 2020-06-03 22:21:47!
----------------------
PROCESSING BATCH 64/199
----------------------


----------------------
PROCESSING BATCH 65/199
----------------------
----------------------
PROCESSING BATCH 66/199
----------------------
----------------------
PROCESSING BATCH 67/199
----------------------
----------------------
PROCESSING BATCH 68/199
----------------------
----------------------
PROCESSING BATCH 69/199
----------------------
